# this notebook is used to generate the vectors based on the diffusor

In [1]:
from datasets import load_dataset, concatenate_datasets, Dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
import datasets
import torch

from transformers import PretrainedConfig
from typing import List
import os
import torch.nn as nn
from transformers import AutoModelForSeq2SeqLM, LEDForConditionalGeneration
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType, PeftModel, PeftModelForSeq2SeqLM, PeftConfig
import torch
from datasets import load_dataset
import os

import argparse
import logging

import boto3
from os import listdir
from os.path import isfile, join
import shutil

from transformers import AutoTokenizer, AutoConfig
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

import sys
sys.path.append('..')
sys.path.append('../src/')
# sys.path.append('../mlti_document_summarization/')
# sys.path.append('../mlti_document_summarization/experiment_scripts/')

# from utils import *
# from data_process import *
import topicmodeling.flanT5_cnn_lighting as fcl

from nltk import sent_tokenize
import pandas as pd

import random
import boto3
import gc

os.environ["TOKENIZERS_PARALLELISM"] = "false"

metric = datasets.load_metric('sacrebleu')

from tqdm import tqdm

#{'translation': {'en': text, 'fr': text}}

/home/wenxiahu/anaconda3/envs/sum-finetune/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workplace/wenxiahu/Vontss/example/../src/topicmodeling/CNN_Encoder.py:15: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('sacrebleu')
Fabric will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Fabric(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.


## defusion in the embedding space

### loading our model

In [2]:
parser = argparse.ArgumentParser()

args = parser.parse_args([])

# args.test_dataset_local_prefix = '/home/wenxiahu/WXWH/long_docs/long_xsum_evaluate_data/1207_t20_1111'

args.finetune_tokenizer_name = 'google/flan-t5-large'
args.finetune_model_name = 'google/flan-t5-large'

args.hidden_size1 = 512
args.max_length = 512

args.device = 'cuda:7'

args.finetune_artifact_local_prefix = '/home/wenxiahu/WXWH/pretrain_reconstruct/model_0122/'
args.finetune_artifact_model_file_name = 'flant5_nest_peftflan-t5-large8'

In [3]:
# config = PeftConfig.from_pretrained(args.finetune_artifact_local_prefix)
# model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, local_files_only=True)
# model = PeftModel.from_pretrained(model, args.finetune_artifact_local_prefix)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# t5autoConfig = T5AutoConfig(
#         hidden_size3=4,
#         model = 'CNN')
# models = FlanT5NestCNNAutoencoder(t5autoConfig).to(args.device)
# print('load', args.finetune_artifact_local_prefix + '/' + args.finetune_artifact_model_file_name)
# logging.info('load ' + args.finetune_artifact_local_prefix + '/' + args.finetune_artifact_model_file_name)
# models.load_state_dict(torch.load(args.finetune_artifact_local_prefix + '/' + args.finetune_artifact_model_file_name))
# models.model = model.to(args.device)
# models.encoder = models.encoder.to(args.device)
# models.decoder = models.decoder.to(args.device)

# models.eval()

In [4]:
# load the model
tokenizer = AutoTokenizer.from_pretrained(args.finetune_tokenizer_name)

config = fcl.T5AutoConfig(hidden_size1=args.hidden_size1, hidden_size3=4, 
                hidden_size2=768, output_size=4 * 768, 
                # model = 'CNN',
                model=args.finetune_model_name,
                )  # Replace with your config
models = fcl.FlanT5NestCNNAutoencoder(config)

print('load', args.finetune_artifact_local_prefix + '/' + args.finetune_artifact_model_file_name)
# logging.info('load ' + args.finetune_artifact_local_prefix + '/' + args.finetune_artifact_model_file_name)
models.load_state_dict(torch.load(args.finetune_artifact_local_prefix + '/' + args.finetune_artifact_model_file_name, map_location='cpu'))

models.model = models.model.to(args.device)
models.encoder = models.encoder.to(args.device)
# models.decoder = models.decoder.to(args.device)

models.eval()

load /home/wenxiahu/WXWH/pretrain_reconstruct/model_0122//flant5_nest_peftflan-t5-large8


FlanT5NestCNNAutoencoder(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
        

### sampling dataset

In [5]:
# dataset_name = 'whu9/xsum_postprocess'
# dataset = load_dataset(dataset_name)
# dataset

In [6]:
# # dataset = load_dataset(dataset_name)

# dataset = dataset['train'].select(range(10000))

In [7]:
# dataset_sample_path = "/home/wenxiahu/WXWH/diffusion/data/data_sample_10000"
# os.makedirs(dataset_sample_path, exist_ok=True)
# dataset.save_to_disk(dataset_sample_path)

### generate the embedding using our model

In [8]:
dataset = load_from_disk("/home/wenxiahu/WXWH/diffusion/data/data_sample_1000")

In [9]:
dataset

Dataset({
    features: ['source', 'summary', 'source_num_tokens', 'summary_num_tokens'],
    num_rows: 1000
})

In [10]:
outputs = []

text_ls = dataset['summary']

batch_size = 2

batch_ls = [text_ls[ind: ind + batch_size]for ind in range(0, len(text_ls), batch_size)]

print(dataset)

for text in tqdm(batch_ls):

    # inputs = tokenizer(text, return_tensors="pt").input_ids
    # attention = tokenizer(text, return_tensors="pt").attention_mask

    # add instruction
    # text = ['repeat: ' + t for t in text]

    inputs = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length = args.max_length)
    
    # get the inputs and attention
    inputs_id = inputs.input_ids.to(models.device)
    attention = inputs.attention_mask.to(models.device)
    
    output = models.model.encoder(inputs_id, attention).last_hidden_state   #batch size * seq length * embedding size, 
    output = models.encoder(output)
    outputs.append(output.detach().cpu())
    
    gc.collect()

Dataset({
    features: ['source', 'summary', 'source_num_tokens', 'summary_num_tokens'],
    num_rows: 1000
})


100%|██████████| 500/500 [02:18<00:00,  3.61it/s]


In [11]:
torch.cat(outputs, dim=0).shape

torch.Size([1000, 4, 1024])

In [12]:
# save the embedding
torch.save(torch.cat(outputs, dim=0), f'./embed_vectors_base_7_1000_prefix.pt')

### train a diffusor using at the embedding space

In [13]:
# run in terminal
# python ./src/diffusion/diffuser_training.py --embedding_input './example/embed_vectors_base_7_1000_prefix.pt' --model_name 'UNet_Conv' --output_dir './example'

### inference

In [16]:
from diffusion.diffuser_training import UNetConv, DDPMScheduler

In [17]:
generated_embeddings = torch.cat(outputs, dim=0)

In [18]:
latent_dim=4 * 1024
num_images=5
device = 'cuda:6'

x = generated_embeddings[:num_images]

model = UNetConv(generated_embeddings.shape[2], generated_embeddings.shape[1], generated_embeddings.shape[1])
model.load_state_dict(torch.load('./diffusion_model.pt'))

model = model.to(device)

model.eval()


UNetConv(
  (embedding): SinusoidalPositionEmbeddings()
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv1d(4, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
          (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), p

In [19]:
x.shape, x.device

(torch.Size([5, 4, 1024]), device(type='cpu'))

### examples: check what sentences we generated

##### denoise from the noise

In [20]:
from diffusion.diffusion_generate import generate_diffused_embed, generate_text

In [21]:
latent_dim, device

(4096, 'cuda:6')

In [22]:
# generate from the noise vector
sampling_turn = 2
timesteps = 1000

x_noise = torch.randn((num_images, 4, latent_dim // 4), device=device)
x_track_ls_ls_noise, x_0_track_ls_ls_noise = generate_diffused_embed(x_noise, model, timesteps, device, batch_size=2, 
                            num_generated_sample=2, return_all_time_embed=True)

Sampling :: 100%|██████████| 1000/1000 [00:05<00:00, 169.27it/s]


In [23]:
# before denoise
x = x_track_ls_ls_noise[0][0]
text_1 = generate_text(x, models, tokenizer, max_length=512, sub_batch=6, device=args.device)
text_1

['[',
 'The US election is being re-elected as a whole by the Labour Party. ',
 'In the US, there are no clear rules on which films can be used. ',
 'In the last couple of weeks, I have been asked by a number of friends to tell me about the history of the Japanese shit. ',
 'In the US, all the races are civil, except for the one where the commutation is prohibited.']

In [24]:
# after denoise
x = x_track_ls_ls_noise[0][-1]
text_1 = generate_text(x, models, tokenizer, max_length=512, sub_batch=6, device=args.device)
text_1

['The United States Postal Service has announced that it will close its offices in the next nine days to allow for the return of its fugitives.',
 'The US election is about to begin. The candidates, who have chosen not to vote, are :',
 "In the course of history, there are no clear winners. But the idea that someone can be a  nuisance '' is being explored by the government.",
 "The US election is about to begin. Here's a list of candidates who have participated in it. ",
 'The US election is a mess. But it is getting better. ']

In [25]:
generate_text(x_noise, models, tokenizer, max_length=512, sub_batch=6, device=args.device)

['The CIA has a new weapon in its bid to retake the throne of London. ',
 'The US election is being re-elected as a whole by the British parliamentary party, which has a history of more than two decades.',
 'In the US, there are no clear rules on which games can be played. ',
 'In the US, there are no clear rules on how long a person can be employed. ',
 'In the US, all the races are civil, except for the one where the ball is being tased by a woman.']